In [ ]:
## training classifier , using topic modelling as features 
## gensim 

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import json
import pandas as pd
import json



In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

C:\Users\Lora\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\Lora\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [4]:
## getting headlines and short_desc together to build a LDA model and then try predicting both 
json_file=r"C:\Users\Lora\Desktop\TxMM\Project\news-category-dataset\News_Category_Dataset_v2.json"

## investigate data 
data = []
for line in open(json_file,'r'):
    data.append(json.loads(line))
    
# Creates DataFrame. 
df = pd.DataFrame(data) 
df= df.drop(['date','authors','link'], axis=1)

In [5]:
# Grouping similar categories in data frame for headline 
df.loc[df['category'] == 'COLLEGE', 'category'] = 'EDUCATION'
df.loc[df['category'] == 'CULTURE & ARTS', 'category'] = 'ARTS'
df.loc[df['category'] == 'ARTS & CULTURE', 'category'] = 'ARTS'
df.loc[df['category'] == 'THE WORLDPOST', 'category'] = 'WORLDPOST'
df.loc[df['category'] == 'WORLD NEWS', 'category'] = 'WORLDPOST'
df.loc[df['category'] == 'GREEN', 'category'] = 'ENVIRONMENT'
df.loc[df['category'] == 'PARENTING', 'category'] = 'PARENTS'
df.loc[df['category'] == 'STYLE & BEAUTY', 'category'] = 'STYLE'


# Dropping categories with two little samples for headline 
df=df.drop(df[df.category == 'FIFTY'].index)
df=df.drop(df[df.category == 'GOOD NEWS'].index)
df=df.drop(df[df.category == 'LATINO VOICES'].index)
df=df.drop(df[df.category == 'MONEY'].index)
df=df.drop(df[df.category == 'WEIRD NEWS'].index)
df=df.drop(df[df.category == 'TASTE'].index)
df=df.drop(df[df.category == 'TECH'].index)
df=df.reset_index(drop=True)

# Dropping empty samples where no short_desc
df.dropna(subset=['short_description'], inplace=True)
df = df.reset_index(drop=True)

# 188 370 samples 

#grouping together
df['Text']=df['headline'] + '. ' + df['short_description']

In [6]:
json_file=r"C:\Users\Lora\Desktop\TxMM\Project\news-category-dataset\News_Category_Dataset_v2.json"

## investigate data 
data = []
for line in open(json_file,'r'):
    data.append(json.loads(line))
    
# Creates DataFrame. 
df_headline = pd.DataFrame(data) 
df_short_desc = pd.DataFrame(data)

# Dropping unnecessary categories from both data frames
df_headline = df_headline.drop(['date','authors','link','short_description'], axis=1)
df_short_desc = df_short_desc.drop(['date','authors','link','headline'], axis=1)

# Grouping similar categories in data frame for headline 
df_headline.loc[df_headline['category'] == 'COLLEGE', 'category'] = 'EDUCATION'
df_headline.loc[df_headline['category'] == 'CULTURE & ARTS', 'category'] = 'ARTS'
df_headline.loc[df_headline['category'] == 'ARTS & CULTURE', 'category'] = 'ARTS'
df_headline.loc[df_headline['category'] == 'THE WORLDPOST', 'category'] = 'WORLDPOST'
df_headline.loc[df_headline['category'] == 'WORLD NEWS', 'category'] = 'WORLDPOST'
df_headline.loc[df_headline['category'] == 'GREEN', 'category'] = 'ENVIRONMENT'
df_headline.loc[df_headline['category'] == 'PARENTING', 'category'] = 'PARENTS'
df_headline.loc[df_headline['category'] == 'STYLE & BEAUTY', 'category'] = 'STYLE'


# Dropping categories with two little samples for headline 
df_headline=df_headline.drop(df_headline[df_headline.category == 'FIFTY'].index)
df_headline=df_headline.drop(df_headline[df_headline.category == 'GOOD NEWS'].index)
df_headline=df_headline.drop(df_headline[df_headline.category == 'LATINO VOICES'].index)
df_headline=df_headline.drop(df_headline[df_headline.category == 'MONEY'].index)
df_headline=df_headline.drop(df_headline[df_headline.category == 'WEIRD NEWS'].index)
df_headline=df_headline.drop(df_headline[df_headline.category == 'TASTE'].index)
df_headline=df_headline.drop(df_headline[df_headline.category == 'TECH'].index)

# Grouping similar categories in data frame for short desc 
df_short_desc.loc[df_short_desc['category'] == 'COLLEGE', 'category'] = 'EDUCATION'
df_short_desc.loc[df_short_desc['category'] == 'CULTURE & ARTS', 'category'] = 'ARTS'
df_short_desc.loc[df_short_desc['category'] == 'ARTS & CULTURE', 'category'] = 'ARTS'
df_short_desc.loc[df_short_desc['category'] == 'THE WORLDPOST', 'category'] = 'WORLDPOST'
df_short_desc.loc[df_short_desc['category'] == 'WORLD NEWS', 'category'] = 'WORLDPOST'
df_short_desc.loc[df_short_desc['category'] == 'GREEN', 'category'] = 'ENVIRONMENT'
df_short_desc.loc[df_short_desc['category'] == 'PARENTING', 'category'] = 'PARENTS'
df_short_desc.loc[df_short_desc['category'] == 'STYLE & BEAUTY', 'category'] = 'STYLE'

# Dropping categories with two little samples for short desc  
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'FIFTY'].index)
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'GOOD NEWS'].index)
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'LATINO VOICES'].index)
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'MONEY'].index)
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'WEIRD NEWS'].index)
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'TASTE'].index)
df_short_desc=df_short_desc.drop(df_short_desc[df_short_desc.category == 'TECH'].index)

# reseting index for both tables
df_headline=df_headline.reset_index(drop=True)
df_short_desc=df_short_desc.reset_index(drop=True)

# checking categories
df_headline.groupby('category').count()
df_short_desc.groupby('category').count()

,short_description
category,
ARTS,3878
BLACK VOICES,4528
BUSINESS,5937
COMEDY,5175
CRIME,3405
DIVORCE,3426
EDUCATION,2148
ENTERTAINMENT,16058
ENVIRONMENT,3945


In [10]:
data_headline = df_headline.headline.values.tolist()
data_short_desc = df_short_desc.short_description.values.tolist()

In [11]:
#Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words_H = list(sent_to_words(data_headline))
data_words_ShD = list(sent_to_words(data_short_desc))

print(data_words_H[:1])
print(data_words_ShD[:1])

[['there', 'were', 'mass', 'shootings', 'in', 'texas', 'last', 'week', 'but', 'only', 'on', 'tv']]
[['she', 'left', 'her', 'husband', 'he', 'killed', 'their', 'children', 'just', 'another', 'day', 'in', 'america']]


In [12]:
###### HEADLINES 

# Build the bigram and trigram models
bigram_headlines= gensim.models.Phrases(data_words_H, min_count=2, threshold=70) # higher threshold fewer phrases.
trigram_headlines = gensim.models.Phrases(bigram_headlines[data_words_H], threshold=70)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_headlines = gensim.models.phrases.Phraser(bigram_headlines)
trigram_mod_headlines = gensim.models.phrases.Phraser(trigram_headlines)

# See trigram example
print(trigram_mod_headlines[bigram_mod_headlines[data_words_H[0]]])

['there', 'were', 'mass_shootings', 'in', 'texas', 'last', 'week', 'but', 'only', 'on', 'tv']


In [13]:
########### SHORT DESCRIPTION

# Build the bigram and trigram models
bigram_SHD= gensim.models.Phrases(data_words_ShD, min_count=2, threshold=100) # higher threshold fewer phrases.
trigram_SHD = gensim.models.Phrases(bigram_SHD[data_words_ShD], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_shd = gensim.models.phrases.Phraser(bigram_SHD)
trigram_mod_shd = gensim.models.phrases.Phraser(trigram_SHD)

# See trigram example
print(trigram_mod_shd[bigram_mod_shd[data_words_ShD[0]]])

NameError: name 'bigram_ShD' is not defined

In [15]:
## USED FOR ALL, HEADLINES, SHORT DESCRIPTION

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod_shd[doc] for doc in texts]  # change bigram depending on what you use all / headlines etc

def make_trigrams(texts):
    return [trigram_mod_shd[bigram_mod_shd[doc]] for doc in texts] # change trigram depending on what you use all / headlines etc

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [16]:
spacy.load('en_core_web_sm')

In [11]:
 ## HEADLINE
# Remove Stop Words
import en_core_web_sm
data_words_nostops = remove_stopwords(data_words_H)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['last', 'week', 'tv']]


In [17]:
 ## SHORT DESC
# Remove Stop Words
import en_core_web_sm
data_words_nostops = remove_stopwords(data_words_ShD)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['leave', 'husband', 'kill', 'child', 'day']]


In [59]:
## ALL 
import en_core_web_sm
data_words_nostops = remove_stopwords(data_words_all)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['leave', 'husband', 'kill', 'child', 'day']]


In [18]:
## ALL 
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [19]:
# The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus.
#Let’s create them.
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)], []]


In [20]:
#Gensim creates a unique id for each word in the document.
#The produced corpus shown above is a mapping of (word_id, word_frequency).
#For example, (0, 1) above implies, word id 0 occurs once in the first document.
#Likewise, word id 1 occurs twice and so on. This is used as the input by the LDA model.
#If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.
id2word[0]

'child'

In [21]:
# https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('child', 1), ('day', 1), ('husband', 1), ('kill', 1), ('leave', 1)]]

In [47]:
# Building the Topic Model
#Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. 
#According to the Gensim docs, both defaults to 1.0/num_topics prior.
#chunksize is the number of documents to be used in each training chunk. 
#update_every determines how often the model parameters should be updated and passes 
#is the total number of training passes.

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True,
                                           )

## save a model 


In [48]:
lda_model.save('descLDA10.model')

In [50]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.081*"take" + 0.066*"love" + 0.038*"even" + 0.038*"work" + 0.027*"still" + '
  '0.023*"start" + 0.020*"couple" + 0.016*"fall" + 0.015*"late" + '
  '0.014*"issue"'),
 (1,
  '0.044*"year" + 0.029*"new" + 0.019*"old" + 0.018*"well" + 0.015*"family" + '
  '0.014*"photo" + 0.014*"call" + 0.013*"wedding" + 0.012*"turn" + '
  '0.011*"man"'),
 (2,
  '0.065*"get" + 0.035*"thing" + 0.029*"much" + 0.025*"little" + 0.021*"seem" '
  '+ 0.020*"change" + 0.020*"food" + 0.019*"watch" + 0.018*"mean" + '
  '0.018*"try"'),
 (3,
  '0.080*"look" + 0.065*"know" + 0.054*"go" + 0.048*"think" + 0.027*"today" + '
  '0.023*"could" + 0.021*"always" + 0.020*"long" + 0.017*"heart" + '
  '0.015*"study"'),
 (4,
  '0.060*"good" + 0.053*"many" + 0.051*"find" + 0.042*"fashion" + 0.041*"show" '
  '+ 0.031*"back" + 0.027*"right" + 0.025*"style" + 0.022*"health" + '
  '0.021*"ask"'),
 (5,
  '0.063*"see" + 0.057*"life" + 0.042*"would" + 0.037*"home" + 0.027*"parent" '
  '+ 0.026*"night" + 0.020*"begin" + 0.020*"exp

In [51]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.112171308402127

Coherence Score:  0.26982644355858376


In [58]:
top_topics

[(0, 0.07528368),
 (1, 0.1530756),
 (2, 0.09425253),
 (3, 0.08608986),
 (4, 0.07328403),
 (5, 0.071904354),
 (6, 0.07692718),
 (7, 0.06345535),
 (8, 0.17731835),
 (9, 0.12840907)]

In [52]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\Lora\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.331983  0.048317       1        1  18.733627
1      0.244973 -0.017661       2        1  15.051244
2     -0.103335  0.427183       3        1   9.958422
3     -0.003004  0.004152       4        1   9.089873
9     -0.024658 -0.123786       5        1   9.069317
6     -0.025900 -0.097152       6        1   8.128135
0     -0.041073 -0.037051       7        1   7.960013
4     -0.116231 -0.096206       8        1   7.719244
5     -0.145893 -0.054147       9        1   7.582522
7     -0.116863 -0.053649      10        1   6.707595, topic_info=     Category          Freq         Term         Total  loglift  logprob
171   Default  12224.000000         make  12224.000000  30.0000  30.0000
92    Default  10965.000000         time  10965.000000  29.0000  29.0000
170   Default  10907.000000         look  10907.000000  28.0000  28.0000
106   Default   9656.000000         take   9656.000000  27.0000  27.0000
38    Default   9675.000000          get   9675.000000  26.0000  26.0000
32    Default   8809.000000          may   8809.000000  25.0000  25.0000
254   Default   8853.000000         know   8853.000000  24.0000  24.0000
530   Default   7890.000000         love   7890.000000  23.0000  23.0000
403   Default   9932.000000         year   9932.000000  22.0000  22.0000
100   Default   7195.000000          see   7195.000000  21.0000  21.0000
1     Default   7514.000000          day   7514.000000  20.0000  20.0000
175   Default   7434.000000           go   7434.000000  19.0000  19.0000
1357  Default   6939.000000         good   6939.000000  18.0000  18.0000
52    Default   6380.000000         come   6380.000000  17.0000  17.0000
718   Default   6531.000000         life   6531.000000  16.0000  16.0000
342   Default   6836.000000         want   6836.000000  15.0000  15.0000
91    Default   6194.000000         many   6194.000000  14.0000  14.0000
182   Default   6579.000000        think   6579.000000  13.0000  13.0000
648   Default   5974.000000         find   5974.000000  12.0000  12.0000
18    Default   8384.000000          say   8384.000000  11.0000  11.0000
450   Default   5161.000000         need   5161.000000  10.0000  10.0000
0     Default   5389.000000        child   5389.000000   9.0000   9.0000
105   Default   7566.000000       people   7566.000000   8.0000   8.0000
465   Default   4838.000000        would   4838.000000   7.0000   7.0000
1626  Default   4853.000000      fashion   4853.000000   6.0000   6.0000
44    Default   4802.000000         show   4802.000000   5.0000   5.0000
34    Default   6485.000000          new   6485.000000   4.0000   4.0000
29    Default   7298.000000         week   7298.000000   3.0000   3.0000
94    Default   5218.000000        thing   5218.000000   2.0000   2.0000
123   Default   4603.000000         even   4603.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
52    Topic10   6379.277344         come   6380.023926   2.7018  -2.7619
293   Topic10   4289.663086        world   4290.409668   2.7018  -3.1588
920   Topic10   3916.884766         help   3917.631592   2.7017  -3.2497
290   Topic10   3036.760986        never   3037.507812   2.7017  -3.5042
643   Topic10   2937.986816       become   2938.733643   2.7017  -3.5373
2186  Topic10   2438.982422        dress   2439.729248   2.7016  -3.7234
2399  Topic10   2409.040771          eat   2409.787598   2.7016  -3.7358
1127  Topic10   2233.697998       report   2234.444824   2.7016  -3.8113
204   Topic10   2119.443115     question   2120.189941   2.7016  -3.8638
2207  Topic10   1865.011963    sometimes   1865.758789   2.7015  -3.9917
292   Topic10   1729.800049     together   1730.546875   2.7015  -4.0670
187   Topic10   1655.229004      believe   1655.975830   2.7015  -4.1111
613   Topic10   1613.720337         body   1614.467041   2.7015  -4.1365
2545  Topic10   1397.270

In [62]:
## trying to obtain feature vectors
train_vecs = []
for i in range(len(df_short_desc)):
    top_topics = lda_model.get_document_topics(corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(10)]
    #topic_vec.extend([df_short_desc.iloc[i].real_counts]) # counts of reviews for restaurant
    #topic_vec.extend([len(df_short_desc.iloc[i].short_description)]) # length review
    train_vecs.append(topic_vec)

In [33]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

In [35]:
from sklearn.metrics import f1_score

In [39]:
from sklearn.linear_model import SGDClassifier

In [46]:
train_vecs

[[0.020086743,
  0.03776424,
  0.030288609,
  0.065002784,
  0.045154218,
  0.07474736,
  0.057647444,
  0.023697134,
  0.04561876,
  0.039016,
  0.05812649,
  0.045166176,
  0.050617665,
  0.044849895,
  0.17550755,
  0.021564182,
  0.04307436,
  0.03745658,
  0.058463912,
  0.026149927,
  76],
 [0.02399561,
  0.045113135,
  0.03618275,
  0.077652276,
  0.053941198,
  0.05037281,
  0.0688656,
  0.02830858,
  0.05449614,
  0.04660849,
  0.06943787,
  0.053955484,
  0.060467828,
  0.053577654,
  0.053982195,
  0.025760558,
  0.0514566,
  0.044745605,
  0.069840945,
  0.031238684,
  24],
 [0.022262758,
  0.041855276,
  0.0335698,
  0.072044596,
  0.050045814,
  0.11895046,
  0.06389245,
  0.026264265,
  0.050560683,
  0.04324264,
  0.06442339,
  0.05005907,
  0.056101125,
  0.049708527,
  0.050083853,
  0.02390025,
  0.047740646,
  0.04151429,
  0.064797364,
  0.028982772,
  87],
 [0.052666783,
  0.03776442,
  0.030288752,
  0.06500309,
  0.04515443,
  0.042167317,
  0.057647716,
  0.023

In [63]:
X = np.array(train_vecs)
y = np.array(df_short_desc.category)

kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        max_iter=4000,
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='weighted'))
    


print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')

C:\Users\Lora\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Lora\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\Lora\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Lora\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
C:\Users\Lora\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\Lora\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:204: UserWarning

Logistic Regression Val f1: 0.054 +- 0.001


C:\Users\Lora\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\Lora\Anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Users\Lora\Anaconda3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Users\Lora\Anaconda3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Users\Lora\Anaconda3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Logisitic Regression SGD Val f1: nan +- nan
SVM Huber Val f1: nan +- nan
